#### Set Up

In [172]:
from pathlib import Path
import json

CLASSIFICATIONS = {
    0: 'none',
    1: 'remove_duplicate_values',
    2: 'fill_missing_values',
    3: 'perform_dimensionality_reduction',
    4: 'perform_correlation_analysis',
}

system_prompts = {f.stem: f.read_text() for f in Path("system_prompt").glob("*.txt")}
system_prompt_classification = system_prompts['classification'].format(functions_dict=str(CLASSIFICATIONS))

user_prompts = [f.read_text() for f in Path("user_prompt").glob("*.txt")]
structured_outputs = {f.stem: json.load(f.open('r')) for f in Path("structured_output").glob("*.json")}
testcases_fill_missing_values = {f.read_text() for f in Path("testcases/fill_missing_values").glob("*.txt")}
testcases_remove_duplicate_values = {f.read_text() for f in Path("testcases/remove_duplicate_values").glob("*.txt")}
testcases_perform_correlation_analysis = {f.read_text() for f in Path("testcases/perform_correlation_analysis").glob("*.txt")}
testcases_perform_dimensionality_reduction = {f.read_text() for f in Path("testcases/perform_dimensionality_reduction").glob("*.txt")}

#### Main System

In [165]:
import re

def postprocess_classification_respond(respond):
    match = re.search(r'\d+', respond)
    if match:
        res = int(match.group(0))
        if res < len(CLASSIFICATIONS) and res > 0:
            return res
    return 0

def perform_classification(client, user_request):
    messages = [
        {
            "role": "system",
            "content": system_prompt_classification,
        },
        {
            "role": "user",
            "content": user_request,
        }
    ]
    response = client.chat.completions.create(
        model="solar-pro",
        messages=messages,
    )
    return postprocess_classification_respond(response.choices[0].message.content)

def perform_function_mapping(client, user_request, system_prompt, structured_output):
    messages = [
            {
                'role': 'system',
                'content': system_prompt
            },
            {
                'role': 'user',
                'content': user_request
            }
        ]
    response = client.chat.completions.create(
            model="solar-pro",
            messages=messages,
            response_format=structured_output
        )
    return response.choices[0].message.content

def process_user_request(client, user_request):
    '''
    Outpupt: function_name (string), function_parameters (JSON), model_respond (string)
    Return the JSON string with the information of the function that the user want to call together the respond of the system.
    '''
    try:
        classification = perform_classification(client, user_request)
        function_name = CLASSIFICATIONS[classification]
        function_parameters = {}
        
        if classification != 0:
            function_parameters = perform_function_mapping(
                client, user_request, 
                system_prompt = system_prompts[CLASSIFICATIONS[classification]], 
                structured_output = structured_outputs[CLASSIFICATIONS[classification]])
            function_parameters = json.loads(function_parameters)
        return True, function_name, function_parameters
    except Exception as e:
        return False, e, {}

In [166]:

import io
import base64
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

def check_cols_in_dataframe(df: pd.DataFrame, cols):
    # Edge Cases
    if cols is None or cols == []:
        return True
    df_columns = df.columns
    
    for col in cols:
        if col not in df_columns:
            return False
    return True

def convert_subset_to_message(subset):
    return subset if subset not in [None, []] else 'all'

'''
The funcitons below return success (bool), message (str), processed_data (pandas.DataFrame), image
'''
def remove_duplicate_values(df: pd.DataFrame, subset, keep):
    if keep not in ['first', 'last', 'false']:
        return False, f'Keep is not recognized: {keep}', None, None
    
    if check_cols_in_dataframe(df,subset):
        try:
            return_message = f'Here is your data with duplicate values remove in subset: {convert_subset_to_message(subset)}, keep: {keep}'
            # String to boolean
            if keep == 'false':
                keep = False
            
            # Subset not specified
            if subset == []:
                subset = None
            
            return True, return_message, df.drop_duplicates(subset=subset, keep=keep), None
        except Exception as e:
            return False, e, None, None
    else:
        return False, f'Columns not in the DataFrame: {subset}', None, None


def fill_missing_values(df: pd.DataFrame, subset, metric):
    if metric not in ['mean', 'median', 'mode']:
        return False, f'Metric is not recognized: {metric}', None, None
    
    if check_cols_in_dataframe(df, subset):
        try:
            df_result = df.copy()
            return_message = f'Here is your data with missing values remove in subset: {convert_subset_to_message(subset)}, metric: {metric}'
            # subset not specified
            if subset == None or subset == []:
                subset = df.select_dtypes(include=np.number).columns.tolist()
            
            for col in subset:
                if metric == 'mean':
                    fill_value = df_result[col].mean()
                elif metric == 'median':
                    fill_value = df_result[col].median()
                elif metric == 'mode':
                    fill_value = df_result[col].mode()
                df_result[col] = df_result[col].fillna(fill_value)
            return True, return_message, df_result, None
        except Exception as e:
            return False, e, None, None
    else:
        return False, f'Columns not in the DataFrame: {subset}', None, None
    
def perform_correlation_analysis(df: pd.DataFrame, subset, method):
    if method not in ['pearson', 'spearman']:
        return False, f"Method is not recognized: {method}", None, None
    
    if check_cols_in_dataframe(df, subset):
        try:
            if subset == None or subset == []:
                subset = df.select_dtypes(include=np.number).columns.tolist()
            
            corr = df[subset].corr(method=method)
            buf = io.BytesIO()
            plt.figure()
            annot_curr = True if len(subset) < 10 else False
            sns.heatmap(corr, annot=annot_curr, cmap='coolwarm', fmt='.2f', square=True, cbar_kws={"shrink": .8})
            plt.title('Correlation Heatmap')
            plt.tight_layout()
            plt.savefig(buf, format='png')
            plt.close()
            buf.seek(0)
            img_base64 = base64.b64encode(buf.read()).decode('utf-8')
            return True, f'Here is the correlation analysis in subset: {convert_subset_to_message(subset)}', corr, img_base64
        except Exception as e:
            return False, e, None, None
    else:
        return False, f'Columns not in the DataFrame: {subset}', None, None
    
def perform_dimensionality_reduction(df: pd.DataFrame, features, target):
    subset = features + [target]
    
    if check_cols_in_dataframe(df, subset):
        try:
            if features == None or features == []:
                features = df.select_dtypes(include=np.number).columns.tolist()
            
            return_message = f'Here is the pca result of features: {features}, target: {target}'
            # PCA
            X = df[features]
            y = df[target]
            pca = PCA(n_components=2)
            X_pca = pca.fit_transform(X)
            pca_df = pd.DataFrame(X_pca, columns=['PC1', 'PC2'], index=df.index)
            
            # Plot
            plt.figure(figsize=(8,6))
            sns.scatterplot(x='PC1', y='PC2', hue=y, palette='viridis', data=pca_df.join(y))
            plt.title('PCA Scatter Plot (PC1 vs PC2)')
            plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]*100:.2f}% variance)')
            plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]*100:.2f}% variance)')
            plt.legend(title=target)
            
            buf = io.BytesIO()
            plt.savefig(buf, format='png')
            plt.close()
            buf.seek(0)
            img_base64 = base64.b64encode(buf.read()).decode('utf-8')
            
            pca_params = {
                'explained_variance': pca.explained_variance_,
                'explained_variance_ratio': pca.explained_variance_.tolist(),
                'components': pca.components_.tolist()
            }
            return True, return_message, pca_params, img_base64 
        except Exception as e:
            return False, e, None, None
    else:
        return False, f'Columns not in the DataFrame: {subset}', None, None
        
def map_json_to_function(df: pd.DataFrame, function_name, function_parameters):
    if function_name == 'remove_duplicate_values':
        return remove_duplicate_values(df, subset=function_parameters['subset'], keep=function_parameters['keep'])
    elif function_name == 'fill_missing_values':
        return fill_missing_values(df, subset=function_parameters['subset'], metric=function_parameters['metric'])
    elif function_name == 'perform_correlation_analysis':
        return perform_correlation_analysis(df, subset=function_parameters['subset'], method=function_parameters['method'])
    elif function_name == 'perform_dimensionality_reduction':
        return perform_dimensionality_reduction(df, features=function_parameters['features'], target=function_parameters['target'])
    else:
        return False, f'No function recognized.', None, None
    

### Testing
#### Load Data

In [167]:
import pandas as pd

train_df = pd.read_csv("../../data/train.csv")
test_df = pd.read_csv("../../data/test.csv")
modified_train_df = pd.read_csv("../../data/modified_train.csv")

#### Set Up

In [169]:
from openai import OpenAI
import toml

parsed_toml = toml.load('../../../.streamlit/secrets.toml')
# API_KEY = st.secrets.get('UPSTAGE_API_KEY', None)
# parsed_toml = toml.load('../../../secrets.toml')

client = OpenAI(
	# api_key=parsed_toml['UPSTAGE_API_KEY'], 
    # api_key=API_KEY,
    api_key=parsed_toml['upstage_api_key_v2'],
 	base_url="https://api.upstage.ai/v1"
)

#### Remove Duplicate Values

In [155]:
outputs = []

for user_prompt in testcases_remove_duplicate_values:
    print(f"user_prompt: {user_prompt}")
    success_function_info, function_name, function_parameters = process_user_request(client, user_request=user_prompt)
    print(f"function_name: {function_name}")
    print(f"function_parameters: {function_parameters}")
    if success_function_info:
        success_request, message, output, image = map_json_to_function(modified_train_df, function_name, function_parameters)
        outputs.append(output)
        print(message)
    print()

user_prompt: Please remove duplicated value in the weather column, keep the last sample of the value.
function_name: remove_duplicate_values
function_parameters: {'keep': 'last', 'subset': ['weather']}
Here is your data with duplicate values remove in subset: ['weather'], keep: last

user_prompt: Please remove duplication in the dataset, keep the first appearance of the value.
function_name: remove_duplicate_values
function_parameters: {'keep': 'first', 'subset': []}
Here is your data with duplicate values remove in subset: all, keep: first

user_prompt: Remove duplication values in columns temp, atemp, and humidity. Don't keep any duplication values.
function_name: remove_duplicate_values
function_parameters: {'keep': 'false', 'subset': ['temp', 'atemp', 'humidity']}
Here is your data with duplicate values remove in subset: ['temp', 'atemp', 'humidity'], keep: false



In [83]:
id = 0
subset = outputs[id].columns

duplications = outputs[id][subset].duplicated()
print(len(duplications[duplications == True]))
outputs[id].head()

0


,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count
0,2011-01-01 00:00:00,1,0,0,1,9.84,14.395,81,0.0,3,13,16
1,2011-01-01 01:00:00,1,0,0,1,9.02,13.635,80,0.0,8,32,40
2,2011-01-01 02:00:00,1,0,0,1,9.02,13.635,80,0.0,5,27,32
3,2011-01-01 03:00:00,1,0,0,1,9.84,14.395,75,0.0,3,10,13
4,2011-01-01 04:00:00,1,0,0,1,9.84,14.395,75,0.0,0,1,1


#### Fill Missing Values

In [58]:
outputs = []

for user_prompt in testcases_fill_missing_values:
    print(f"user_prompt: {user_prompt}")
    success_function_info, function_name, function_parameters = process_user_request(client, user_request=user_prompt)
    print(f"function_name: {function_name}")
    print(f"function_parameters: {function_parameters}")
    if success_function_info:
        success_request, message, output, image = map_json_to_function(modified_train_df, function_name, function_parameters)
        outputs.append(output)
        print(message)
    print()

user_prompt: Fill the missing values with mean.
function_name: fill_missing_values
function_parameters: {'metric': 'mean', 'subset': []}
Here is your data with missing values remove in subset: all, metric: mean

user_prompt: Fill the NA values with the column's most frequent value in 'windspeed'.
function_name: Error code: 500 - {'error': {'message': 'internal_server_error', 'type': 'internal_server_error', 'param': None, 'code': None}}
function_parameters: {}

user_prompt: Fill the NULL values with median in class 'temp' and 'atemp'.
function_name: Error code: 500 - {'error': {'message': 'internal_server_error', 'type': 'internal_server_error', 'param': None, 'code': None}}
function_parameters: {}



In [59]:
id = 0
rows_with_nan = outputs[id].loc[outputs[id].isna().any(axis=1)]
rows_with_nan

,datetime,season,holiday,workingday,weather,temp,atemp,humidity,windspeed,casual,registered,count


#### Perform Correlation Analysis

In [136]:
outputs = []
images = []

for user_prompt in testcases_perform_correlation_analysis:
    print(f"user_prompt: {user_prompt}")
    success_function_info, function_name, function_parameters = process_user_request(client, user_request=user_prompt)
    print(f"function_name: {function_name}")
    print(f"function_parameters: {function_parameters}")
    if success_function_info:
        success_request, message, output, image = map_json_to_function(train_df, function_name, function_parameters)
        outputs.append(output)
        images.append(image)
        print(message)
    print()

user_prompt: Give me the heatmap of the entire database with standard pearson correlation coefficient.
function_name: perform_correlation_analysis
function_parameters: {'method': 'pearson', 'subset': []}
Here is the correlation analysis in subset: ['season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count']

user_prompt: Execute correlation analysis on columns weather, atemp, temp, and humidity with the metric spearman rank correlation.
function_name: perform_correlation_analysis
function_parameters: {'method': 'spearman', 'subset': ['weather', 'atemp', 'temp', 'humidity']}
Here is the correlation analysis in subset: ['weather', 'atemp', 'temp', 'humidity']



In [139]:
id = 1

print(outputs[id])

           weather     atemp      temp  humidity
weather   1.000000 -0.061933 -0.057912  0.399492
atemp    -0.061933  1.000000  0.987128 -0.042028
temp     -0.057912  0.987128  1.000000 -0.046854
humidity  0.399492 -0.042028 -0.046854  1.000000


In [138]:
from PIL import Image

for id, image in enumerate(images):
    img = Image.open(io.BytesIO(base64.decodebytes(bytes(image, "utf-8"))))
    if img.mode in ("RGBA", "P"):
        img = img.convert("RGB")
    img.save(f'../../data/corr_{id}.jpeg')

#### Perform Dimensionality Reduction

In [173]:
outputs = []
images = []

for user_prompt in testcases_perform_dimensionality_reduction:
    print(f"user_prompt: {user_prompt}")
    success_function_info, function_name, function_parameters = process_user_request(client, user_request=user_prompt)
    print(f"function_name: {function_name}")
    print(f"function_parameters: {function_parameters}")
    if success_function_info:
        success_request, message, output, image = map_json_to_function(train_df, function_name, function_parameters)
        outputs.append(output)
        images.append(image)
        print(message)
    print()

user_prompt: I want you to perform PCA with the target column weather.
function_name: perform_dimensionality_reduction
function_parameters: {'features': [], 'target': 'weather'}
Here is the pca result of features: ['season', 'holiday', 'workingday', 'weather', 'temp', 'atemp', 'humidity', 'windspeed', 'casual', 'registered', 'count'], target: weather

user_prompt: I want you to excecute dimensionality reduction with target colum: season with features: temp, atemp, humidity, and windspeed.

function_name: Error code: 500 - {'error': {'message': 'internal_server_error', 'type': 'internal_server_error', 'param': None, 'code': None}}
function_parameters: {}



In [174]:
outputs[0]

{'explained_variance': array([55871.57393504,  2333.22480586]),
 'explained_variance_ratio': [55871.573935036846, 2333.2248058608875],
 'components': [[0.0007759861504612084,
   -7.302777989881548e-06,
   9.05256815162328e-05,
   -0.00033275700486311803,
   0.0123123913871312,
   0.013230045725711497,
   -0.024810042883131784,
   0.00343235079884392,
   0.1340059051060151,
   0.6301731215536966,
   0.7641790266597118],
  [-0.0003547572485204797,
   0.000221851890345421,
   -0.004285006755954742,
   -0.0011459370560865797,
   0.04953853973057426,
   0.053258703294930626,
   -0.09239960857314228,
   0.008201758763864439,
   0.7992977644486559,
   -0.5174819488671101,
   0.28181581558154595]]}

In [175]:
from PIL import Image

for id, image in enumerate(images):
    img = Image.open(io.BytesIO(base64.decodebytes(bytes(image, "utf-8"))))
    if img.mode in ("RGBA", "P"):
        img = img.convert("RGB")
    img.save(f'../../data/pca_{id}.jpeg')